# Edge-AI

In [ ]:
import datetime
import json
import pathlib

import httpx

edge_ai_url = "http://localhost:8000"

In [ ]:
prompt = "The book, The Ministry for the Future, by Kim Stanley Robinson published in 2020 by Orbit, New York"

In [ ]:
result = httpx.post(f"{edge_ai_url}/inventory/instance/generate",
                    json={"text": prompt})

In [ ]:
result.json()

In [ ]:
second_prompt = "Please catalog the book titled The Dark Archive by Genevieve Cogman published in 2020 by ACE in New York"

In [ ]:
second_result = httpx.post(f"{edge_ai_url}/inventory/instance/generate",
                    json={"text": second_prompt})

In [ ]:
second_result.json()

In [ ]:
second_result.json()['instance']

In [ ]:
third_prompt = "Please catalog Better Homes & Garden's Salad Book published in 1958 in New York"

In [ ]:
third_result = httpx.post(f"{edge_ai_url}/inventory/instance/generate",
                    json={"text": third_prompt})

In [ ]:
third_result

In [ ]:
third_result.json()

In [ ]:
second_result.json()

## Indexing Records in RAG index

In [ ]:
records_path = pathlib.Path(
    "local-instance-2024-08-22T00:52:49.671780+00:00.jsonl"
)

In [ ]:
result = httpx.post(
    f"{edge_ai_url}/inventory/instance/index",
    json={"source": str(records_path.absolute()) },
    timeout=60.
)

In [ ]:
result

In [ ]:
result.json()

In [ ]:
index_path = pathlib.Path("/Users/jpnelson/40-49 AI and Machine Learning/41.02 FOLIO AI Edge Module/.ragatouille/colbert/indexes/Instances")

In [ ]:
index_path.exists()

In [ ]:
result = httpx.post(
    f"{edge_ai_url}/inventory/instance/similarity",
    json={
        "text": {'title': 'The Ministry for the Future',
 'source': 'FOLIO',
 'instanceTypeText': 'text',
 'contributors': [{'name': 'Kim Stanley Robinson',
   'contributorTypeText': 'Author'}],
 'publication': [{'publisher': 'Orbit',
   'dateOfPublication': '2020',
   'place': 'New York'}]}},
    timeout=60
)

In [ ]:
result

In [ ]:
result.json()

In [ ]:
existing_instance = json.loads('{"id": "bab80eca-e29c-459c-a8ea-8c282311f155", "_version": "3", "hrid": "in2333978", "source": "MARC", "title": "Chronik der christlichen Welt ...", "administrativeNotes": [], "indexTitle": "Chronik der christlichen Welt ...", "parentInstances": [], "childInstances": [], "isBoundWith": false, "alternativeTitles": [], "editions": [], "series": [], "identifiers": [{"value": "(ICU)BID21257283", "identifierTypeText": "System Control Number"}, {"value": "(OCoLC)191709788", "identifierTypeText": "OCLC"}], "contributors": [], "subjects": [{"authorityId": null, "value": "Lutheran church--Periodicals."}, {"authorityId": null, "value": "Periodicals."}], "classifications": [], "publication": [{"publisher": "J.C.B. Mohr", "place": "Leipzig [etc.]", "dateOfPublication": null, "role": null}], "publicationFrequency": [], "publicationRange": ["Ceased with v. 27 (1917)."], "electronicAccess": [], "instanceFormatIds": [], "physicalDescriptions": ["v. 34 cm."], "languages": ["ger"], "notes": [], "modeOfIssuanceId": "363895f3-3c52-421e-8f78-9ec105ec15f1", "previouslyHeld": false, "discoverySuppress": false, "statisticalCodeIds": ["05dfeb83-c186-433e-aa98-a8372b73c5b8"], "statusId": "319fefa7-92bd-4d45-9746-4614555955c6", "statusUpdatedDate": "2019-12-18T22:11:08Z", "tags": {"tagList": []}, "natureOfContentTermIds": [], "precedingTitles": [], "succeedingTitles": [], "instanceTypeText": "text"}')

In [ ]:
result2 = httpx.post(
    f"{edge_ai_url}/inventory/instance/similarity",
    json={
        "text": json.loads(result.json()['instance'])
    },
    timeout=60
)

In [ ]:
result2.json()

## Find Similarity with Instance RAG

In [ ]:
cookbook = json.loads(third_result.json()['instance'])

In [ ]:
result = httpx.post(f"{edge_ai_url}/inventory/instance/similarity",
                    json={
                        "text": cookbook
                    },
                    timeout=60)

In [ ]:
result.json()

## Index 10k Instances 


In [ ]:
result = httpx.post(f"{edge_ai_url}/inventory/instance/index/start?limit=10000&offset=4000000")

In [ ]:
result.json()

## Index 30k (by 10k each)

In [ ]:
start = datetime.datetime.now(datetime.UTC)
print(f"Starting at {start}")
dag_runs = []
for offset in [3_775_000]: # 10_000, 150_000, 1_120_000,]: 2_250_000, 3_550_000, 3_775_000, 4_740_000, 5_910_000, 6_010_000, 7_840_000]:
    print(f"\tStart offset {offset:,}")
    result = httpx.post(f"{edge_ai_url}/inventory/instance/index/start?limit=10000&offset={offset}")
    print(f"DAG run ID {result.json()['dag_run_id']}")
    dag_runs.append(result.json())
end = datetime.datetime.now(datetime.UTC)
print(f"Finished at {end} Total time {(end-start).seconds:,} seconds")